## 介绍 Introduction

这个笔记是非常基础和简单的介绍融合模型的入门方式，特别是堆叠融合的变体。简而言之，堆叠用作第一级（基础），一些基本的机器学习模型（分类器）的预测，然后在第二级使用另一个模型来预测早期一级预测的输出。

This notebook is a very basic and simple introductory primer to the method of ensembling models, in particular the variant of ensembling known as Stacking. In a nutshell stacking uses as a first-level (base), the predictions of a few basic machine learning models (classifiers) and then uses another model at the second-level to predict the output from the earlier first-level predictions.

泰坦尼克号数据集是引入这一概念的主要方案，许多新手到Kaggle是从这里开始的。此外，即使堆叠技术已经为许多团队在Kaggle比赛获胜的方案，似乎在这个话题上缺少核心，所以我希望这款笔记本可以填补一些空白。

The Titanic dataset is a prime candidate for introducing this concept as many newcomers to Kaggle start out here. Furthermore even though stacking has been responsible for many a team winning Kaggle competitions there seems to be a dearth of kernels on this topic so I hope this notebook can fill somewhat of that void.

我自己也是Kaggle的新手，我设法学习的第一个合适的融合/堆叠脚本是由伟大的Faron在AllState Severity索赔比赛中写的。这笔记的教材大量借鉴了Faron的脚本，尽管我们是融合分类，他的是融合回归方案。但无论如何，请在这里查看他的脚本：

I myself am quite a newcomer to the Kaggle scene as well and the first proper ensembling/stacking script that I managed to chance upon and study was one written in the AllState Severity Claims competition by the great Faron. The material in this notebook borrows heavily from Faron's script although ported to factor in ensembles of classifiers whilst his was ensembles of regressors. Anyway please check out his script here:

[Stacking Starter](https://www.kaggle.com/mmueller/allstate-claims-severity/stacking-starter/run/390867) : by Faron

现在在笔记本上我希望它能以一种直观和简洁的方式，做到公正并且传达出融合的概念。我的另一个独立的Kaggle脚本实现了完全相同的组合步骤（尽管有不同的参数），下面讨论的公共LB分数为0.808，这足以达到前9％，运行时间不到4分钟。 因此，我很确定有很多改进的空间，并添加到该脚本。 无论如何，请随时给我留下任何关于我如何改善的意见

Now onto the notebook at hand and I hope that it manages to do justice and convey the concept of ensembling in an intuitive and concise manner. My other standalone Kaggle [script](https://www.kaggle.com/arthurtok/titanic/simple-stacking-with-xgboost-0-808) which implements exactly the same ensembling steps (albeit with different parameters) discussed below gives a Public LB score of 0.808 which is good enough to get to the top 9% and runs just under 4 minutes. Therefore I am pretty sure there is a lot of room to improve and add on to that script. Anyways please feel free to leave me any comments with regards to how I can improve

In [170]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;

## 特征探索，工程和清洁 Feature Exploration, Engineering and Cleaning

现在我们将进行常规的工作，就像大多数要点的结构一样，首先是探索手头的数据，找出可能的特征工程，以及对任何分类特征进行数字编码。

Now we will proceed much like how most kernels in general are structured, and that is to first explore the data on hand, identify possible feature engineering opportunities as well as numerically encode any categorical features.

In [171]:
# Load in the train and test datasets
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')
print(train.shape)
print(test.shape)
# Store our passenger ID for easy access
PassengerId = test['PassengerId']

(891, 12)
(418, 11)


In [172]:
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


毫无疑问，我们的任务是以某种方式从分类变量中提取信息。

Well it is no surprise that our task is to somehow extract the information out of the categorical variables

**特征工程**

**Feature Engineering**

在这里，必须扩展到Sina的特征工程理念，这是非常全面和周全的笔记，所以请看看他的工作

Here, credit must be extended to Sina's very comprehensive and well-thought out notebook for the feature engineering ideas so please check out his work

[Titanic Best Working Classfier](https://www.kaggle.com/sinakhorami/titanic/titanic-best-working-classifier): by Sina

In [173]:
full_data = [train, test]

# Some features of my own that I have added in
# Gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Feature engineering steps taken from Sina
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
# Create a New feature CategoricalAge
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)
# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] ;

In [174]:
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,CategoricalFare,CategoricalAge,Title
0,1,0,3,"Braund, Mr. Owen Harris",1,1,1,0,A/5 21171,0,NaN,0,23,0,2,0,"(-0.001, 7.91]","(16.0, 32.0]",1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,PC 17599,3,C85,1,51,1,2,0,"(31.0, 512.329]","(32.0, 48.0]",3


In [175]:
test.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,Title
0,892,3,"Kelly, Mr. James",1,2,0,0,330911,0,NaN,2,16,0,1,1,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,2,1,0,363272,0,NaN,0,32,0,2,0,3


In [176]:
# # 特征选择train[["Survived"
# # 准备训练和测试数据

# print(train.shape)
# y = train["Survived"]
# print("y.shape",y.shape)

# # print(train.shape)
# tf_train = train.drop(["Survived"], axis = 1)
# drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
# tf_train = tf_train.drop(drop_elements, axis = 1)
# tf_train = tf_train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
# print(X_train.shape)
# # X_train_ss_new = SelectKBest(f_classif, k=16).fit_transform(X_train, y)
# # print(X_train_ss_new.shape)
# X_train.head(2)

# # X_train.to_csv('./out1.csv')


# tf_train_new_pd = pd.DataFrame(tf_train_new)
# tf_train_new_pd.to_csv('./out2.csv')

# # 独热编码
# y = (np.arange(2) == y[:,None]).astype(np.float32)
# print("y.shape",y.shape)
# print("tf_train_new.shape",tf_train_new.shape)
# print("tf_labels.shape",tf_labels.shape)
# # 'Pclass','Sex','Parch','Fare','Embarked','Name_length','Has_Cabin','IsAlone','Title'

In [177]:
# print(tf_labels.shape)
# print(type(tf_labels))
# tf_labels.head(5)

In [178]:
# 特征选择
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']


y = train["Survived"]

tf_train = train.drop(["Survived"], axis = 1)
tf_train = tf_train.drop(drop_elements, axis = 1)
tf_train = tf_train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)

train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)


test  = test.drop(drop_elements, axis = 1)


# from sklearn.feature_selection import SelectKBest,f_classif
# tf_train_new = SelectKBest(f_classif, k=9).fit_transform(tf_train, y)
# tf_test = test[['Pclass','Sex','Parch','Fare','Embarked','Name_length','Has_Cabin','IsAlone','Title']]

In [179]:
train.head(3)

print('train.shape',train.shape)
print('test.shape',test.shape)
print('type(train)',type(train))
print('type(test)',type(test))

train.shape (891, 12)
test.shape (418, 11)
type(train) <class 'pandas.core.frame.DataFrame'>
type(test) <class 'pandas.core.frame.DataFrame'>


In [180]:
# 用训练，测试，目标创建的Numpy数组提供给模型
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models

y_train = train['Survived'].ravel()
train = train.drop(['Survived'], axis=1)
x_train = train.values # Creates an array of the train data
x_test = test.values # Creats an array of the test data



In [181]:
print("x_train.shape",x_train.shape)
print("y_train.shape",y_train.shape)
print("train.shape",train.shape)
print("x_test.shape",x_test.shape)
print('type(x_train)',type(x_train))
print('type(x_test)',type(x_test))
print('type(train)',type(train))
print('type(test)',type(test))

x_train.shape (891, 11)
y_train.shape (891,)
train.shape (891, 11)
x_test.shape (418, 11)
type(x_train) <class 'numpy.ndarray'>
type(x_test) <class 'numpy.ndarray'>
type(train) <class 'pandas.core.frame.DataFrame'>
type(test) <class 'pandas.core.frame.DataFrame'>


In [70]:
# x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, svc_oof_train), axis=1)
# x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, svc_oof_test), axis=1)

现在连接和加入第一级训练和测试预测集是x_train和x_test，我们现在能放入第二级训练模型

Having now concatenated and joined both the first-level train and test predictions as x_train and x_test, we can now fit a second-level learning model.

In [154]:
test.head(2)


,Pclass,Sex,Age,Parch,Fare,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,Title
0,3,1,2,0,0,2,16,0,1,1,1
1,3,0,2,0,0,0,32,0,2,0,3


In [71]:
# 计算精准度
# def accuracy(predictions, labels):
#   return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
#           / predictions.shape[0])
# 保存结果
# def savecsv(test_prediction_np, filename="submission_tf_titanic.csv"):
#     np.savetxt('submission_tf_titanic.csv', 
#            np.c_[range(1,len(x_test)+1),test_prediction_np], 
#            delimiter=',', 
#            header = 'PassengerId,Survived', 
#            comments = '', 
#            fmt='%d')
#     StackingSubmission = pd.DataFrame({'PassengerId': PassengerId,'Survived': test_prediction_np })
#     StackingSubmission.to_csv(filename, index=False)

In [72]:
print(x_train.shape)
print(y_train.shape)
print(train.shape)
train.head(2)
# y_train

(891, 11)
(891,)
(891, 11)


,Pclass,Sex,Age,Parch,Fare,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,Title
0,3,1,1,0,0,0,23,0,2,0,1
1,1,0,2,0,3,1,51,1,2,0,3


In [182]:
# 独热编码
y_train = (np.arange(2) == y_train[:,None]).astype(np.float32)
print(y_train.shape)

(891, 2)


In [183]:
#形成验证数据
train_dataset, valid_dataset = model_selection.train_test_split(train, test_size=0.3, random_state=0)
train_labels, valid_labels = model_selection.train_test_split(y_train, test_size=0.3, random_state=0)
print(train_dataset.shape)
print(valid_dataset.shape)
print(train_labels.shape)
print(valid_labels.shape)

(623, 11)
(268, 11)
(623, 2)
(268, 2)


In [118]:
# 2. 定义超参数和placeholder
learning_rate = 0.5
epochs = 10
batch_size = 100

# placeholder
# 原始数据的维度：9
x = tf.placeholder(tf.float32, [None, 11])
# 输出为0-1的one-hot编码
y = tf.placeholder(tf.float32, [None, 2])

In [119]:
# 3. 定义参数w和b
# hidden layer => w, b
W1 = tf.Variable(tf.random_normal([11, 22], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([22]), name='b1')
# output layer => w, b
W2 = tf.Variable(tf.random_normal([22, 2], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([2]), name='b2')

In [120]:
# 4. 构造隐层网络
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

In [121]:
# 5. 构造输出（预测值）
# 计算输出
y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))

In [122]:
# 6. BP部分—定义loss 
# 对n个标签计算交叉熵
# 对m个样本取平均
y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped) + (1 - y) * tf.log(1 - y_clipped), axis=1))

In [123]:
# 7. BP部分—定义优化算法
# 创建优化器，确定优化目标
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

In [124]:
# 8. 定义初始化operation和准确率node
# init operator
init_op = tf.global_variables_initializer()

# 创建准确率节点
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [125]:
# 9 开始训练
# 创建session
print("tf_train_new.shape",tf_train_new.shape)
print("y.shape",y.shape)
with tf.Session() as sess:
    # 变量初始化
    sess.run(init_op)
    total_batch = int(len(x_train) / batch_size)
    print(total_batch)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
#             batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            _, c = sess.run([optimizer, cross_entropy], feed_dict={x: train_dataset, y: train_labels})
#     sess.run(train_step, feed_dict={xs: tf_train_new, ys: y, keep_prob: 0.7})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost = ", "{:.3f}".format(avg_cost))
    print("accuracy",sess.run(accuracy, feed_dict={x: valid_dataset, y: valid_labels}))

tf_train_new.shape (891, 9)
y.shape (?, 2)
8
Epoch: 1 cost =  21.076
Epoch: 2 cost =  23.831
Epoch: 3 cost =  23.831
Epoch: 4 cost =  23.831
Epoch: 5 cost =  23.831
Epoch: 6 cost =  23.831
Epoch: 7 cost =  23.831
Epoch: 8 cost =  23.831
Epoch: 9 cost =  23.831
Epoch: 10 cost =  23.831
accuracy 0.37313432


In [184]:
x = tf.placeholder(tf.float32,shape = [None,11],name = 'input')
y = tf.placeholder(tf.float32,shape = [None,2],name = 'label')
weights1 = tf.Variable(tf.random_normal([11,11]),name = 'weights1')
bias1 = tf.Variable(tf.zeros([11]),name = 'bias1')
a = tf.nn.relu(tf.matmul(x,weights1) + bias1)
weights2 = tf.Variable(tf.random_normal([11,2]),name = 'weights2')
bias2 = tf.Variable(tf.zeros([2]),name = 'bias2')
z = tf.matmul(a,weights2) + bias2
y_pred = tf.nn.softmax(z)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=z))
correct_pred = tf.equal(tf.argmax(y,1),tf.argmax(y_pred,1))
acc_op = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
train_op = tf.train.AdamOptimizer(0.001).minimize(cost)

In [187]:
import os
from sklearn.ensemble import RandomForestRegressor
# 存档入口
saver = tf.train.Saver()

# 在Saver声明之后定义的变量将不会被存储
# non_storable_variable = tf.Variable(777)

ckpt_dir = './ckpt_dir'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

with tf.Session() as sess:
    tf.global_variables_initializer().run()

    ckpt = tf.train.latest_checkpoint(ckpt_dir)
    if ckpt:
        print('Restoring from checkpoint: %s' % ckpt)
        saver.restore(sess, ckpt)

    for epoch in range(30):
        total_loss = 0.
        for i in range(len(x_train)):
            feed_dict = {x: [x_train[i]],y:[y_train[i]]}
            _,loss = sess.run([train_op,cost],feed_dict=feed_dict)
            total_loss +=loss
        print('Epoch: %4d, total loss = %.12f' % (epoch,total_loss))
        if epoch % 10 == 0:
            accuracy = sess.run(acc_op,feed_dict={x:valid_dataset,y:valid_labels})
            print("Accuracy on validation set: %.9f" % accuracy)
            saver.save(sess, ckpt_dir + '/logistic.ckpt')
    print('training complete!')

    accuracy = sess.run(acc_op,feed_dict={x:valid_dataset,y:valid_labels})
    print("Accuracy on validation set: %.9f" % accuracy)
    pred = sess.run(y_pred,feed_dict={x:valid_dataset})
    correct = np.equal(np.argmax(pred,1),np.argmax(valid_labels,1))
    numpy_accuracy = np.mean(correct.astype(np.float32))
    print("Accuracy on validation set (numpy): %.9f" % numpy_accuracy)

    saver.save(sess, ckpt_dir + '/logistic.ckpt')
    
    
    '''
    测试数据的清洗和训练数据一样，两者可以共同完成
    '''
    

    
    # 读测试数据  
    test_data = pd.read_csv('./input/test.csv')  

    #数据清洗, 数据预处理  
    test_data.loc[test_data['Sex']=='male','Sex'] = 0
    test_data.loc[test_data['Sex']=='female','Sex'] = 1 

    age = test_data[['Age','Sex','Parch','SibSp','Pclass']]
    age_notnull = age.loc[(test_data.Age.notnull())]
    age_isnull = age.loc[(test_data.Age.isnull())]
    X = age_notnull.values[:,1:]
    Y = age_notnull.values[:,0]
    rfr = RandomForestRegressor(n_estimators=1000,n_jobs=-1)
    rfr.fit(X,Y)
    predictAges = rfr.predict(age_isnull.values[:,1:])
    test_data.loc[(test_data.Age.isnull()),'Age'] = predictAges

    test_data['Embarked'] = test_data['Embarked'].fillna('S')
    test_data.loc[test_data['Embarked'] == 'S','Embarked'] = 0
    test_data.loc[test_data['Embarked'] == 'C','Embarked'] = 1
    test_data.loc[test_data['Embarked'] == 'Q','Embarked'] = 2

    test_data.drop(['Cabin'],axis=1,inplace=True)

    #特征选择  
    X_test = test_data[['Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare']]  
    print('x_test.shape',x_test.shape)
    print('X_test.shape',X_test.shape)
    print('test.shape',test.shape)
    
    print('type(x_test)',type(x_test))
    print('type(X_test)',type(X_test))
    print('type(test)',type(test))
#     print(x_test)
#     print(X_test)
    
#     评估模型  
    predictions = np.argmax(sess.run(y_pred, feed_dict={x: test}), 1)  

    #保存结果  
    submission = pd.DataFrame({  
        "PassengerId": test_data["PassengerId"],  
        "Survived": predictions  
    })  
    submission.to_csv("titanic-submission.csv", index=False)  

Restoring from checkpoint: ./ckpt_dir/logistic.ckpt
INFO:tensorflow:Restoring parameters from ./ckpt_dir/logistic.ckpt
Epoch:    0, total loss = 443.474575500899
Accuracy on validation set: 0.794776142
Epoch:    1, total loss = 442.164824451611
Epoch:    2, total loss = 440.860925284144
Epoch:    3, total loss = 439.851188335454
Epoch:    4, total loss = 439.304571123397
Epoch:    5, total loss = 438.516713683077
Epoch:    6, total loss = 437.851598415404
Epoch:    7, total loss = 437.402197927748
Epoch:    8, total loss = 436.649372247949
Epoch:    9, total loss = 436.313598281255
Epoch:   10, total loss = 436.223107103812
Accuracy on validation set: 0.802238822
Epoch:   11, total loss = 436.664002979633
Epoch:   12, total loss = 436.219860768551
Epoch:   13, total loss = 436.032932988287
Epoch:   14, total loss = 435.878378415659
Epoch:   15, total loss = 434.239313399410
Epoch:   16, total loss = 434.418905993610
Epoch:   17, total loss = 432.998642806640
Epoch:   18, total loss = 4